cv2 is an abbreviation for "OpenCV", which stands for Open Source Computer Vision Library. It is a popular open-source computer vision and machine learning software library used for real-time image and video processing. cv2 is a Python library that provides a Python interface to the OpenCV C++ library. It provides various functions and tools for image and video manipulation, feature detection, object recognition, and more. cv2 is widely used in academic research, industry, and hobbyist projects for a variety of applications such as robotics, surveillance, augmented reality, and medical imaging.

In [1]:
# Import necessary libraries
import cv2
import pickle
import cvzone
import numpy as np

In [ ]:
#Video feed
# Create a VideoCapture object to read from the input video file
cap = cv2.VideoCapture('carPark.mp4')

# Load the parking space positions from a saved file
with open('CarParkPos','rb') as f:
        posList= pickle.load(f)

# Set the dimensions of each parking space
width, height = 107, 48

# Define a function to check whether a parking space is free or occupied
def checkParkingSpace(imgPro):
    
     # Initialize a counter for the number of free parking spaces
     spaceCounter = 0
    
     # Loop over each parking space position
     for pos in posList:
        # Get the x and y coordinates of the current parking space
        x,y = pos
            
        # Crop the input image to the current parking space    
        imgCrop = imgPro[y:y + height, x:x + width]
        #cv2.imshow(str(x * y), imgCrop) #to show all diff images that we have, we need to have unique name for img, hence str(x*y)
        # Count the number of non-zero pixels in the cropped image
        count = cv2.countNonZero(imgCrop)
        
         #If the count is less than a threshold, the space is considered free
        if count < 900:
            color = (0,255,0) # green color for free parking space
            thickness =  5
            spaceCounter +=1
            # Otherwise, the space is considered occupied
        else:
            color = (0,0,255) # red color for occupied parking space
            thickness = 2
            
        # Draw a rectangle around the current parking space with the corresponding color
        cv2.rectangle(img,pos,(pos[0] + width,pos[1] + height),color,2)
        
     # Add a text label showing the number of free parking spaces out of the total number of spaces   
     cvzone.putTextRect(img,f'Free Spaces:{spaceCounter}/{len(posList)}', (100, 50), scale=3, thickness=5, offset=20, colorR=(0,200,0))           
    
    
# Loop over each frame in the input video    
while True:
    
    # If we have reached the end of the video, start over from the beginning
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT): #CAP_PROP_POS_FRAMES-Gives us total frames we have in video
        cap.set(cv2.CAP_PROP_POS_FRAMES,0)
        
    # Read the next frame from the video
    success, img= cap.read()
    
    # Convert the image to grayscale and apply Gaussian blur to reduce noise
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 1)
    
    # Apply adaptive thresholding to obtain a binary image with the parking spaces highlighted
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        cv2.THRESH_BINARY_INV,25,16)
    
    # Apply median blur to further reduce noise
    imgMedian = cv2.medianBlur(imgThreshold,5)
    
    # Apply dilation to increase the thickness of the white borders around the parking spaces
    kernel = np.ones((3,3),np.uint8)
    imgDilate = cv2.dilate(imgMedian,kernel, iterations=1)
    
    # Call the function to check each parking space and update the image accordingly
    checkParkingSpace(imgDilate)

    # Show the output image
    cv2.imshow("Image",img)
    
    #cv2.imshow("ImageBlur",imgBlur)
    #cv2.imshow("ImageThresh",imgThreshold)
    
    # Apply median blur to further reduce noise
    #cv2.imshow("ImageMedian",imgMedian)
    #cv2.imshow("ImageDilate",imgDilate) #to increase the thickness of white borders
    cv2.waitKey(10)  #10 to slow down the video